In [ ]:
%pip install accelerate

In [ ]:
%pip install transformers

In [ ]:
%pip install pandas

In [ ]:
%pip install matplotlib.pyplot

In [ ]:
%pip install numpy

In [ ]:
%pip install torch

In [ ]:
%pip install numpy scipy scikit-learn

In [76]:
from sklearn.metrics import f1_score

In [8]:
%pip install -q evaluate

Note: you may need to restart the kernel to use updated packages.


In [9]:
import torch
import torch.nn as nn
import torch.optim as optim
import accelerate
import transformers as trfm
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from transformers import Trainer, TrainingArguments
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import pandas as pd

In [10]:
tokenizer = AutoTokenizer.from_pretrained('distilroberta-base')
model = AutoModelForSequenceClassification.from_pretrained('distilroberta-base')

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at distilroberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.out_proj.bias', 'classifier.dense.weight', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
from datasets import Dataset

In [12]:
dfa = pd.read_csv('NLP3/dfa.csv')

In [13]:
dfa.size

10042

In [14]:
train, test = train_test_split(dfa, shuffle=True,  random_state=34)

In [15]:
data_dict = train.to_dict(orient='list')
datatrain = Dataset.from_dict(data_dict)

In [16]:
d_dict = test.to_dict(orient='list')
datatest = Dataset.from_dict(d_dict)
print(datatest[:10])

{'label': [1, 0, 0, 0, 0, 1, 0, 0, 0, 0], 'sms': ['Bored housewives! Old world chat n date immediately! 0871750. 77. 11! BT - internal rate 10p / fukkianese just from land line!', 'Moji i love you more than words. Have a rich day\n', "Sorry, I'll call later In meeting.\n", 'Oh... Lk tt den we take e one tt ends at cine lor... Dun wan yogasana oso can... \n', 'Gettin rdy to ship comp\n', 'URGENT! We are trying to contact U. Todays draw shows that you have won a £2000 prize GUARANTEED. Call 09066358361 from land line. Claim Y87. Valid 12hrs only\n', 'I am in bus on the way to calicut\n', 'S da..al r above  &lt;#&gt;\n', "I'll text carlos and let you know, hang on\n", 'Only just got this message, not ignoring you. Yes, i was. Shopping that is\n']}


In [41]:
def tokenize_function(examples):
    return tokenizer(examples['sms'], padding='max_length', truncation=True, max_length = 512)
train = datatrain.map(tokenize_function, batched=True)
train
test = datatest.map(tokenize_function, batched=True)
test

Map:   0%|          | 0/3765 [00:00<?, ? examples/s]

Map:   0%|          | 0/1256 [00:00<?, ? examples/s]

Dataset({
    features: ['label', 'sms', 'input_ids', 'attention_mask'],
    num_rows: 1256
})

In [42]:
train = train.rename_column('label', 'labels')

In [43]:
torch.tensor(train['labels'])

tensor([0, 1, 0,  ..., 0, 0, 1])

In [44]:
test = test.rename_column('label', 'labels')

In [45]:
torch.tensor(test['labels'])

tensor([1, 0, 0,  ..., 0, 0, 0])

In [46]:
class_weights = torch.tensor(1 - (dfa["label"].value_counts().sort_index() / len(dfa))).float()
class_weights

tensor([0.2976, 0.7024])

In [47]:
train

Dataset({
    features: ['labels', 'sms', 'input_ids', 'attention_mask'],
    num_rows: 3765
})

In [48]:
test_outputs = test['labels']
test_inputs = test.data.drop(columns=['labels'])

In [22]:
class Trainer(Trainer):
  def compute_loss(self, model, inputs, return_outputs=False):
    outputs = model(**inputs)
    logits = outputs.get("logits")
    labels = inputs.get ("labels")
    loss_func = nn.CrossEntropyLoss(class_weights)
    loss = loss_func(logits, labels)
    return (loss, outputs) if return_outputs else loss

In [79]:
# from skilearn.metrix import f1_score
import numpy as np
# metric = evaluate.load("f1")

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    f1 = f1_score(labels, preds)
    return {'f1': f1}

In [24]:
output_dir = "outputs"
training_args = TrainingArguments(output_dir=output_dir
)

In [25]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train,
    # eval_dataset=test
)

In [26]:
trainer.train()

Step,Training Loss
500,0.131100
1000,0.031900


TrainOutput(global_step=1413, training_loss=0.0585005429309054, metrics={'train_runtime': 29997.6364, 'train_samples_per_second': 0.377, 'train_steps_per_second': 0.047, 'total_flos': 1496219267819520.0, 'train_loss': 0.0585005429309054, 'epoch': 3.0})

In [116]:
pred = trainer.predict(test)

In [117]:
pred

PredictionOutput(predictions=array([[-5.1329694,  5.4082756],
       [ 5.149126 , -5.178597 ],
       [ 5.1462445, -5.1613593],
       ...,
       [ 5.147734 , -5.1762652],
       [ 5.15749  , -5.1746306],
       [ 5.1480155, -5.160688 ]], dtype=float32), label_ids=array([1, 0, 0, ..., 0, 0, 0]), metrics={'test_loss': 0.0443299300968647, 'test_runtime': 1007.6626, 'test_samples_per_second': 1.246, 'test_steps_per_second': 0.156})

In [83]:
from sklearn.metrics import classification_report

In [118]:
classification_report_trans = classification_report(test['labels'], np.argmax(pred.predictions, axis=-1))

In [119]:
print(classification_report_trans)

              precision    recall  f1-score   support

           0       1.00      0.99      0.99       879
           1       0.99      0.99      0.99       377

    accuracy                           0.99      1256
   macro avg       0.99      0.99      0.99      1256
weighted avg       0.99      0.99      0.99      1256



In [120]:
compute_metrics(pred)

{'f1': 0.9880794701986755}

In [152]:
# pipeline = Pipeline(model=trainer, )
# sample = "thanks for all your answers guys i know i should have checked the rsync manual but i would rather ge..."
# sample = 'You’ve been selected to receive a free gift!'
# sample = "Congratulations! You've been selected for a free vacation. Click here to claim your prize and enjoy a luxury getaway to a tropical paradise. Don't miss out!"
# sample = "Hello [Your Name], we're excited to inform you that you've won a complimentary vacation package to a beautiful destination. Click the link to redeem your prize and start planning your dream getaway. We can't wait for you to experience the luxury and relaxation of this amazing trip!"
# pipe("You’ve been selected to receive a free gift!")
# a = "You’ve been selected to receive a free gift!"
from transformers import TextClassificationPipeline
pipe = TextClassificationPipeline(model=model, tokenizer=tokenizer)
input_text = 'You’ve been selected to receive a free gift!'
pipe(input_text)

[{'label': 'LABEL_1', 'score': 0.9999212026596069}]